In [1]:
import os
import json
import operator
from typing import Annotated, Any, Dict, List, Optional, Union, TypedDict
import getpass

# LangChain / LangGraph Imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

# --- API KEY SETUP ---
# If you haven't set this in your environment variables, uncomment below:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [2]:
# --- HELPER FUNCTIONS ---

def find_next_missing_field(data: Dict, path: List[str] = []) -> Optional[Dict]:
    """
    Recursively searches for the first field where 'requis' is True
    and 'valeur' is empty/None.
    """
    for key, value in data.items():
        current_path = path + [key]

        if isinstance(value, dict) and "valeur" in value and "requis" in value:
            if value.get("type") == "fixe":
                continue
            
            # Check if value is empty
            is_empty = value["valeur"] in [None, "", []]
            if value["requis"] and is_empty:
                return {"path": current_path, "schema": value, "key": key}

        elif isinstance(value, dict):
            result = find_next_missing_field(value, current_path)
            if result:
                return result

        elif isinstance(value, list):
            for idx, item in enumerate(value):
                if isinstance(item, dict):
                    result = find_next_missing_field(item, current_path + [str(idx)])
                    if result:
                        return result
    return None

def update_json_value(data: Dict, path: List[str], new_value: Any):
    """Updates the JSON structure at the specific path in-place."""
    ref = data
    for key in path[:-1]:
        if key.isdigit() and isinstance(ref, list):
            ref = ref[int(key)]
        else:
            ref = ref[key]

    last_key = path[-1]
    if last_key.isdigit() and isinstance(ref, list):
        ref[int(last_key)]["valeur"] = new_value
    else:
        ref[last_key]["valeur"] = new_value

In [3]:
# --- STATE DEFINITION ---

class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    contract_data: Dict
    current_field_info: Optional[Dict]
    completed: bool

# --- NODES ---

def determine_progress(state: AgentState):
    data = state.get("contract_data")
    next_field = find_next_missing_field(data)

    if not next_field:
        return {"completed": True, "current_field_info": None}

    return {"completed": False, "current_field_info": next_field}

def generate_question(state: AgentState):
    field_info = state["current_field_info"]
    path_str = " > ".join(field_info["path"])
    field_type = field_info["schema"].get("type")
    options = field_info["schema"].get("options", [])

    system_prompt = f"""
    You are a helpful French real estate legal assistant filling out a rental contract.
    Your goal is to ask the user for specific information to fill the JSON field: "{path_str}".
    
    Field Type: {field_type}
    Options: {options}
    
    Phrase your question naturally in French.
    """
    
    # specific instruction to handle the very first turn if messages are empty
    msg = llm.invoke([SystemMessage(content=system_prompt)])
    return {"messages": [msg]}

# Re-run this cell to update the logic
def process_answer(state: AgentState):
    messages = state["messages"]
    last_user_msg = messages[-1]
    field_info = state["current_field_info"]

    if not field_info:
        return {}

    # --- FIX 1: Use Union instead of Any ---
    # OpenAI needs concrete types. We allow the most common ones.
    class Extraction(BaseModel):
        value: Union[str, int, float, bool] = Field(
            description="The extracted value. Use correct type (bool for booleen, int/float for nombre)."
        )

    field_type = field_info["schema"].get("type")
    options = field_info["schema"].get("options", [])

    extraction_prompt = f"""
    Extract the value for field "{field_info['key']}" from: "{last_user_msg.content}"
    
    Target Type: {field_type}
    Allowed Options: {options}
    
    - If type is 'booleen', return a boolean.
    - If type is 'nombre', return a number.
    - If type is 'date', return a string YYYY-MM-DD.
    """

    # --- FIX 2: Specify method="function_calling" ---
    # This creates a more compatible schema for Unions and optional fields
    structured_llm = llm.with_structured_output(Extraction, method="function_calling")
    
    result = structured_llm.invoke(extraction_prompt)

    # Update the main JSON data
    current_data = state["contract_data"]
    update_json_value(current_data, field_info["path"], result.value)

    return {"contract_data": current_data}
def save_contract(state: AgentState):
    # Saving to a local file in the notebook directory
    with open("contrat_finalise.json", "w", encoding="utf-8") as f:
        json.dump(state["contract_data"], f, ensure_ascii=False, indent=2)
    
    return {
        "messages": [AIMessage(content="Terminé ! Le contrat a été sauvegardé dans 'contrat_finalise.json'.")]
    }

# --- GRAPH CONSTRUCTION ---

workflow = StateGraph(AgentState)

workflow.add_node("determine_progress", determine_progress)
workflow.add_node("generate_question", generate_question)
workflow.add_node("process_answer", process_answer)
workflow.add_node("save_contract", save_contract)

def route_start(state: AgentState):
    # If last message is from Human, we need to process it
    if state["messages"] and isinstance(state["messages"][-1], HumanMessage):
        return "process_answer"
    return "determine_progress"

def route_after_progress(state: AgentState):
    if state["completed"]:
        return "save_contract"
    return "generate_question"

workflow.set_conditional_entry_point(
    route_start,
    {"process_answer": "process_answer", "determine_progress": "determine_progress"},
)

workflow.add_edge("process_answer", "determine_progress")
workflow.add_conditional_edges(
    "determine_progress",
    route_after_progress,
    {"save_contract": "save_contract", "generate_question": "generate_question"},
)
workflow.add_edge("generate_question", END)
workflow.add_edge("save_contract", END)

app = workflow.compile()

In [4]:
# 1. Define a Mock Template for the Contract
initial_contract_template = {
    "bailleur": {
        "nom_complet": {"valeur": None, "requis": True, "type": "texte"},
        "adresse": {"valeur": None, "requis": True, "type": "texte"}
    },
    "locataire": {
        "nom_complet": {"valeur": None, "requis": True, "type": "texte"}
    },
    "details_logement": {
        "type_bien": {"valeur": None, "requis": True, "type": "texte", "options": ["Appartement", "Maison"]},
        "surface_m2": {"valeur": None, "requis": True, "type": "nombre"},
        "meuble": {"valeur": None, "requis": True, "type": "booleen"}
    }
}

def run_interactive_session():
    print("--- 📝 Assistant Juridique Immobilier (Mode Notebook) ---")
    print("Type 'quit' to exit.\n")

    # Initialize State
    current_state = {
        "messages": [],
        "contract_data": initial_contract_template,
        "current_field_info": None,
        "completed": False
    }

    # Start the conversation loop
    while True:
        # Run the graph until it stops (at END)
        # We pass the full current configuration to maintain state
        events = app.invoke(current_state)
        
        # Update our local state variable with the result from the graph
        current_state = events
        
        # Get the last message generated by the AI
        last_message = current_state["messages"][-1]
        
        # Print AI Response
        print(f"\n🤖 Assistant: {last_message.content}")

        # Check if we are done
        if current_state.get("completed", False):
            print("\n✅ Process Finished.")
            break
        
        # Get User Input
        user_input = input("\n👤 Vous: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            break
            
        # Append User Input to messages so the graph sees it next run
        current_state["messages"].append(HumanMessage(content=user_input))

# Run the interactive loop
run_interactive_session()

--- 📝 Assistant Juridique Immobilier (Mode Notebook) ---
Type 'quit' to exit.


🤖 Assistant: Quel est le nom complet du bailleur ?

🤖 Assistant: Quelle est l'adresse complète du bailleur, s'il vous plaît ?

🤖 Assistant: Quel est le nom complet du locataire ?

🤖 Assistant: Quel est le type de bien que vous souhaitez louer ? Est-ce un appartement ou une maison ?

🤖 Assistant: Quelle est la surface en mètres carrés du logement que vous souhaitez louer ?

🤖 Assistant: Le logement est-il meublé ? (Oui/Non)

🤖 Assistant: Terminé ! Le contrat a été sauvegardé dans 'contrat_finalise.json'.

✅ Process Finished.


In [5]:
from IPython.display import Image, display

print("--- Attempting to Draw Graph ---")

try:
    # This requires internet access (calls mermaid.ink API) and 'grandalf' for layout
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"\n❌ Image generation failed. Error details:\n{e}")
    
    print("\n--- Fallback: ASCII Representation ---")
    try:
        app.get_graph().print_ascii()
    except Exception as e_ascii:
        print(f"Could not print ASCII either: {e_ascii}")

--- Attempting to Draw Graph ---

❌ Image generation failed. Error details:
Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 500.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

--- Fallback: ASCII Representation ---
                   +-----------+                   
                   | __start__ |                   
                   +-----------+                   
                 ...            ...                
               ..                  ..              
             ..                      ..            
  +----------------+                   ..          
  | process_answer |                 ..            
  +----------------+               ..              
      